In [1]:
# credits: https://towardsdatascience.com/pyspark-and-xgboost-integration-tested-on-the-kaggle-titanic-dataset-4e75a568bdb

In [2]:
cd ..

/home/hantoine/Documents/Cours/Concordia/2019-Winter/SOEN_691_Big_Data_Analytics/project/accident-prediction-montreal


In [3]:
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.sql.types import *
from pyspark.sql.functions import col
import numpy as np
from pyspark.ml.feature import StringIndexer, VectorAssembler
import os
from utils import init_spark
from preprocess import get_positive_samples, \
                       get_negative_samples, \
                       get_dataset_df
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars data/xgboost4j-spark-0.72.jar,data/xgboost4j-0.72.jar pyspark-shell'

In [4]:
spark = init_spark()
spark.sparkContext.addPyFile("data/sparkxgb.zip")
from sparkxgb import XGBoostEstimator

Spark Session created
Parameters:
	spark.driver.extraClassPath: ./data/xgboost4j-spark-0.72.jar:./data/xgboost4j-0.72.jar
	spark.network.timeout: 300s
	spark.master: local[10]
	spark.executor.id: driver
	spark.driver.port: 46015
	spark.driver.host: laptop-hantoine.wireless.concordia.ca
	spark.cleaner.periodicGC.interval: 5min
	spark.app.name: Accident prediction
	spark.serializer: org.apache.spark.serializer.KryoSerializer
	spark.driver.memory: 6g
	spark.rdd.compress: True
	spark.app.id: local-1556568635624
	spark.serializer.objectStreamReset: 100
	spark.submit.deployMode: client
	spark.ui.showConsoleProgress: true
	spark.jars: file:///home/hantoine/Documents/Cours/Concordia/2019-Winter/SOEN_691_Big_Data_Analytics/project/accident-prediction-montreal/data/xgboost4j-spark-0.72.jar,file:///home/hantoine/Documents/Cours/Concordia/2019-Winter/SOEN_691_Big_Data_Analytics/project/accident-prediction-montreal/data/xgboost4j-0.72.jar
	spark.repl.local.jars: file:///home/hantoine/Documents/Cour

In [5]:
# load dataset
neg_samples = get_negative_samples(spark).sample(0.5).na.fill(-999)
pos_samples = get_positive_samples(spark).sample(1.0).na.fill(-999)
(neg_samples.count(), pos_samples.count())

(2318466, 134489)

In [6]:
imbalance_ratio = neg_samples.count()/pos_samples.count()

In [7]:
train, test = get_dataset_df(spark, pos_samples, neg_samples)

Skip extracting road features: already done


In [14]:
#create xgboost estimator
xgboost = XGBoostEstimator(
    featuresCol="features", 
    labelCol="label", 
    predictionCol="prediction",
    scale_pos_weight=imbalance_ratio,
    max_depth=30,
    eta=0.27,
    gamma=45.0,
    subsample=1.0,
    colsample_bytree=0.95,
    alpha=0.6
)

In [15]:
model = xgboost.fit(train)

In [16]:
from pyspark.sql.functions import col
test_pred = (model
             .transform(test)
             .withColumn('rawPrediction',
                         col('probabilities'))
             .withColumnRenamed('probabilities',
                                'probability'))
train_pred = (model
              .transform(train)
              .withColumn('rawPrediction',
                          col('probabilities'))
              .withColumnRenamed('probabilities',
                                 'probability'))

In [17]:
from export_results import *
result_dir = create_result_dir('xgb')
write_params(model, neg_samples.count(), result_dir)
write_results(test_pred, train_pred, result_dir)

Area Under PR = 0.5261814470035253
Area Under ROC = 0.909525275356034
Area Under PR = 0.6714504302062363
Area Under ROC = 0.977258305926859
